In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
plt.style.use('default')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap
import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines()) 
import os


# Aula 17: Classificação de Textos com Redes Neurais
**Objetivo: ao fim desta aula, o aluno será capaz de programar, treinar e interpretar uma rede neural para classificação de textos usando Keras**


In [ ]:
df = pd.read_csv('./datasets/IMDB Dataset.csv')
vectorizer = CountVectorizer(binary=True, stop_words=stopwords, max_features=1000, max_df=0.4)
X = vectorizer.fit_transform(list(df['review'])).toarray()
print(X.shape)

# Exercício 1
*Objetivo: analisar um código e suas saídas para entender o que é one-hot encoding*

Uma das maneiras de codificar uma saída é através do processo one-hot encoding. Analise o código abaixo e responda:

1. Como funciona o "one-hot encoding"?
1. O que significa cada linha da matriz de one-hot encodings?
1. O que significa cada coluna da matriz de one-hot encodings?



In [ ]:
from sklearn.preprocessing import OneHotEncoder
labels = np.array([list(df['sentiment'])]).T
ohe = OneHotEncoder()
y = ohe.fit_transform(labels).toarray()
print(type(y))
print(y.shape)
print(labels[0:5,:])
print(y[0:5,:])


# Exercício 2
*Objetivo: relacionar a teoria de redes neurais a sua implementação para classificação de textos*

A primeira rede neural que implementaremos tem uma entrada X, na mesma forma que estávamos usando no sklearn. A rede neural implementa a equação:

$$
y=\sigma(XW)
$$

Onde:
* $y$ é a saída,
* $X$ é a entrada,
* $W$ é uma matriz
* $\sigma( . )$ é uma função sigmoide

A ideia da rede é ter tantas saídas quantas são as possíveis classes em nossos dados. A saída deve ser igual a 1 para a classe “correta”, e 0 para todas as outras classes.

1. No caso do IMDB dataset, quantas saídas devemos ter em nossa rede?
1. No código abaixo, existe a função `rede_neural_simples`. Ela define um modelo de rede neural. Encontre onde definimos a existência da função sigmoide, e onde definimos que existirá a multiplicação matricial $XW$.
1. Lembrando que $X$ é a matriz que marca a presença de uma palavra em cada documento, o que significam as dimensões de W?
1. Mais adiante, fazemos o plot da rede. O que significa cada campo desse plot?
1. O que a função que define a rede neural retorna? A rede neural executa assim que a função é chamada?



In [ ]:
from keras.models import Model
from keras.layers import Input, Dense

In [ ]:
def rede_neural_simples(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  y = Dense(n_dims_out, activation='sigmoid', name='classificador')(x)
  return Model(input_layer, y)

In [ ]:
from tensorflow.keras.utils import plot_model

rede_neural = rede_neural_simples(X.shape[1], y.shape[1])
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

# Exercício 3
*Objetivo: treinar uma rede neural simples e analisar as curvas de loss*

Execute o processo de treino (fit) da rede neural simples.

1. O que é Early Stopping? Por que ele é relevante?
1. O que significa dizer que a métrica de erro é `mse` (mean squared error)?
1. O que significa dizer que o otimizador é `adam` (adaptive momentum)?
1. O que significa o parâmetro `validation_split`?
1. O que significa o parâmetro `patience`?
1. Como as curvas de `loss` e `val_loss` se comportam? O que isso significa?

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

NameError: name 'X' is not defined

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.show()

# Exercício 4
*Objetivo: analisar o desempenho da rede neural*

Analise o código abaixo, para avaliação da rede neural

1. Por que precisamos usar o método `ohe.inverse_transform()` nas saídas da rede neural para usar o `classification_report`?
1. Como o desempenho da rede neural se compara com o desempenho das outras estratégias de classificação que já usamos nesta disciplina?



In [ ]:
y_est = rede_neural.predict(X_test)
print(y_est[0:5,:])
print(y_test[0:5,:])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

# Exercício 5
*Objetivo: analisar os pesos da rede neural*

Voltando à equação da rede neural:

$$
y=\sigma(XW)
$$

1. Quantas linhas e colunas deve ter a matriz W?
1. O que significa cada elemento $w_{i,j}$?
1. Como poderíamos usar a matriz W para entender o quanto cada palavra foi relevante em nosso processo de classificação?
1. Verifique o código da seção “que palavras têm mais peso?”. Ele implementa uma ideia parecida com a que você propôs?
1. Verifique as palavras que foram encontradas como mais relevantes para a classificação. Esse resultado concorda com o que tivemos para os classificadores anteriores?



In [ ]:
w = rede_neural.get_layer('classificador').get_weights()
#print(w)
#print(w[0])

In [ ]:
# Visualização 1: quais são as palavras de maior peso?
pesos = np.abs(w[0][:,0] - w[0][:,1])
feature_names = vectorizer.get_feature_names() # Modificar isso para versões mais atuais do sklearn
pares = [ (pesos[i], feature_names[i]) for i in range(len(feature_names))]
pares = sorted(pares, reverse=True)
pesos_ = [c[0] for c in pares]
palavras_ = [c[1] for c in pares]

n_palavras = 50
plt.figure(figsize=(14,3))
plt.bar(np.arange(n_palavras), pesos_[0:n_palavras])
plt.xticks(np.arange(n_palavras), palavras_[0:n_palavras], rotation=80)
plt.xlabel('Palavras')
plt.ylabel('Peso no classificador')
plt.show()

# Exercício 6
*Objetivo: entender como realizar uma projeção intermediária*

Analise o código abaixo, da função `rede_neural_proj`. Ele implementa uma rede neural um pouco diferente da anterior.

1. Qual é essa diferença?
1. Como ela poderia ser escrita matematicamente (complementando a equação do caso anterior)?
1. Houve diferença no desempenho da rede ao adicionar a nova camada?

In [ ]:
def rede_neural_proj(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Dense(2, name='projecao')(x)
  y = Dense(n_dims_out, activation='sigmoid', name='classificador')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_proj(X.shape[1], y.shape[1])
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

# Exercício 7
*Objetivo: interpretar o espaço latente gerado por uma rede neural com uma camada oculta*

1. Quais são as dimensões da matriz da camada `projeção` de nossa rede neural? O que ela está projetando?
1. O que você esperaria ver nesse espaço vetorial interno da rede – também chamado de `espaço latente`?
1. Analise o código que mostra as projeções intermediárias da rede neural. O que ele mostra? Como devemos interpretá-lo?



In [ ]:
# Visualização 2: onde foi parar cada palavra?
v = rede_neural.get_layer('projecao').get_weights()[0]
plt.figure(figsize=(4,4))
plt.scatter(v[:,0], v[:,1], s=1, alpha=0.3, c='b')
for s in ["director", "actor", "bad", "good", "excellent", "plot", "worst", "terrible", "waste", "awful", "fantastic"]:
    _n = vectorizer.vocabulary_[s]
    plt.text(v[_n,0], v[_n,1], s, ha='center')
plt.title('Projeção das palavras no espaço latente')
plt.ylabel('Componente 2')
plt.xlabel('Componente 1')
#plt.xlim([-20,20])
#plt.ylim([-20,20])
plt.show()

# Exercício 8
*Objetivo: analisar o espaço latente quando aumentamos o tamanho da dimensão latente da rede neural*

Se a dimensão do espaço latente da rede neural for maior que dois, é necessário usar alguma técnica de redução de dimensionalidade, como `PCA`, para visualizá-la.

O que acontece com as palavras no espaço latente e com o desempenho da rede quando aumentamos a dimensão do espaço latente?